In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os

In [2]:
# import train_csv

import pandas as pd
classes = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')

In [3]:
#split the various tags in train_csv
classes['splitted tags'] = classes['tags'].apply(lambda x: x.split())


In [4]:
# combine all tags into a set to know the unique number of tags
combine_list = []
for i, n in enumerate(classes['splitted tags']):
    combine_list.extend(n)
combine_set = set(combine_list) 
len(combine_set)

17

In [5]:
# visualize how frequent the different labels occur

# from collections import Counter
# count_dic = Counter(combine_list)
# count_df = pd.DataFrame(count_dic, index = ['counts']).T
# plt.figure(figsize=(10, 8))
# sns.barplot(x=count_df.index, y= count_df['counts'])
# plt.xticks(rotation= 45, fontsize = 13)

In [6]:
# building the train_labels
columns = sorted(list(combine_set))
train_labels = pd.DataFrame(np.zeros((40479, 17)), columns = columns)
train_labels

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
for n, lists in tqdm(enumerate(classes['splitted tags'])):
    for column in lists:
        train_labels[column][n]=1


40479it [00:28, 1416.39it/s]


In [8]:
train_labels['image_name'] = classes['image_name'] + '.jpg'
train_labels

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,image_name
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,train_0.jpg
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,train_1.jpg
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_2.jpg
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_3.jpg
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,train_4.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_40474.jpg
40475,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train_40475.jpg
40476,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_40476.jpg
40477,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,train_40477.jpg


In [9]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.models import Model

In [10]:
# defining the metric value function
from keras import backend

def fbeta(y_true, y_pred, beta=2):
    #clip predictions
    y_pred = backend.clip(y_pred, 0, 1)
    # calculate elements
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis = 1)
    fp = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis = 1)
    fn = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis = 1)
    
    # calculate precision
    p = tp/(tp + fp + backend.epsilon())
    #calculate recall
    r = tp / (tp + fn + backend.epsilon())
    #calculate fbeta
    bb = beta ** 2
    fbeta_score = backend.mean((1+ bb) * (p * r)/ (bb * p + r + backend.epsilon()))
    return fbeta_score

In [11]:
train_datagen = ImageDataGenerator(featurewise_center = True,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  rotation_range = 90)
val_datagen = ImageDataGenerator(featurewise_center = True)
train_datagen.mean = [123.68, 116.779, 103.939]
val_datagen.mean= [123.68, 116.779, 103.939]


In [12]:
columns

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [13]:
# creating a scaled iterator ti feed into model
train_it = train_datagen.flow_from_dataframe(train_labels[: 30479], 
                                             directory = '../input/planets-dataset/planet/planet/train-jpg', 
                                             x_col = 'image_name', y_col = columns, 
                                             class_mode= 'raw', batch_size = 64, target_size = (128, 128))
val_it = val_datagen.flow_from_dataframe(train_labels[30479:], 
                          directory = '../input/planets-dataset/planet/planet/train-jpg', 
                          x_col = 'image_name', 
                          y_col = columns,
                          class_mode= 'raw', 
                          batch_size = 64, target_size = (128, 128))


Found 30479 validated image filenames.
Found 10000 validated image filenames.


In [14]:
len(train_it)

477

In [15]:
import gc
gc.collect()

62

In [16]:

# model = Sequential()
# model.add(Conv2D(32, (3,3), padding = 'same', input_shape=(128, 128, 3)))
# model.add(Conv2D(32, (3,3), padding = 'same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(64, (3,3), padding='same'))
# model.add(Conv2D(64, (3,3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Conv2D(128, (3,3), padding='same'))
# model.add(Conv2D(128, (3,3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Flatten())

# model.add(Dense(128))
# model.add(Activation('relu'))

# model.add(Dense(17))
# model.add(Activation('sigmoid'))
# opt = SGD(lr=0.01, momentum=0.9)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[fbeta])

# history = model.fit_generator(train_it, steps_per_epoch = len(train_it), epochs = 30, 
#                               validation_data= val_it, validation_steps = len(val_it))

In [17]:
# load model
model = VGG16(include_top = False, input_shape= (128, 128, 3))

# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
    
# allow las vgg block to be trainable
model.get_layer('block5_conv1').trainable = True
model.get_layer('block5_conv2').trainable = True
model.get_layer('block5_conv3').trainable = True
model.get_layer('block5_pool').trainable= True

# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu')(flat1)
output = Dense(17, activation='sigmoid')(class1)

# define new model
model = Model(inputs=model.inputs, outputs=output)

# compile model
model.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics = [fbeta])

58892288/58889256 [==============================] - 0s 0us/step


In [18]:
history = model.fit_generator(train_it, steps_per_epoch = len(train_it), validation_data = val_it, 
                             validation_steps = len(val_it), epochs = 25)

Epoch 1/50
477/477 [==============================] - 228s 478ms/step - loss: 0.2122 - fbeta: 0.7926 - val_loss: 0.1665 - val_fbeta: 0.8512
Epoch 2/50
477/477 [==============================] - 207s 433ms/step - loss: 0.1552 - fbeta: 0.8587 - val_loss: 0.1448 - val_fbeta: 0.8763
Epoch 3/50
477/477 [==============================] - 206s 431ms/step - loss: 0.1389 - fbeta: 0.8760 - val_loss: 0.1321 - val_fbeta: 0.8849
Epoch 4/50
477/477 [==============================] - 207s 435ms/step - loss: 0.1316 - fbeta: 0.8828 - val_loss: 0.1282 - val_fbeta: 0.8855
Epoch 5/50
477/477 [==============================] - 209s 439ms/step - loss: 0.1256 - fbeta: 0.8894 - val_loss: 0.1210 - val_fbeta: 0.8905
Epoch 6/50
477/477 [==============================] - 207s 433ms/step - loss: 0.1215 - fbeta: 0.8910 - val_loss: 0.1202 - val_fbeta: 0.8995
Epoch 7/50
477/477 [==============================] - 205s 431ms/step - loss: 0.1191 - fbeta: 0.8941 - val_loss: 0.1164 - val_fbeta: 0.8965
Epoch 8/50
477/477 [

KeyboardInterrupt: 

In [ ]:
model.save('semifinal_model.h5')

In [ ]:
import gc
gc.collect()

In [19]:
import numpy as np
test_images = []
for img1 in os.listdir('../input/planets-dataset/planet/planet/test-jpg'):
    test_images.append('../input/planets-dataset/planet/planet/test-jpg/{}'.format(img1))
for image in os.listdir('../input/planets-dataset/test-jpg-additional/test-jpg-additional'):
    test_images.append('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}'.format(image))

In [20]:
test_labels = pd.DataFrame(test_images, columns = ['image_names'])
test_labels

,image_names
0,../input/planets-dataset/planet/planet/test-jp...
1,../input/planets-dataset/planet/planet/test-jp...
2,../input/planets-dataset/planet/planet/test-jp...
3,../input/planets-dataset/planet/planet/test-jp...
4,../input/planets-dataset/planet/planet/test-jp...
...,...
61186,../input/planets-dataset/test-jpg-additional/t...
61187,../input/planets-dataset/test-jpg-additional/t...
61188,../input/planets-dataset/test-jpg-additional/t...
61189,../input/planets-dataset/test-jpg-additional/t...


In [21]:
test_datagen = ImageDataGenerator(featurewise_center= True)
test_datagen.mean = [123.68, 116.779, 103.939]
test_generator= test_datagen.flow_from_dataframe(test_labels, directory = None,
                                                 x_col = 'image_names', 
                                                 y_col = None, batch_size = 64, 
                                                 class_mode=None, target_size=(128, 128), 
                                                 shuffle = False)

Found 61191 validated image filenames.


In [22]:
test_generator.reset()
pred = model.predict_generator(test_generator, steps = len(test_generator))

In [23]:
pred = pred.round()

In [24]:
predictions = pd.DataFrame(pred, columns = columns)

In [25]:
predictions['image_name'] = test_labels['image_names'].apply(lambda x: x.split('/')[-1])

In [26]:
def get_columns_ones(df, n):
    d = dict(zip(df.columns, list(df.iloc[n])))
    # extract keys of dict with ones
    l = [k for k, v in d.items() if v == 1.0]
    result = ' '.join(l)
    return result

In [27]:
predictions2 = predictions.copy()
predictions2['tags'] = 0
for i in predictions2.index:
    predictions2['tags'][i] = get_columns_ones(predictions, i)
predictions2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,image_name,tags
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,test_16476.jpg,agriculture clear primary
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,test_15164.jpg,clear primary
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,test_16984.jpg,agriculture clear primary
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,test_4415.jpg,partly_cloudy primary
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,test_5227.jpg,agriculture clear habitation primary road
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,file_12356.jpg,clear primary
61187,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,file_7191.jpg,agriculture clear habitation primary road
61188,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,file_17520.jpg,agriculture clear primary road
61189,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,file_4931.jpg,agriculture clear habitation primary road


In [28]:
predictions2 = predictions2[['image_name', 'tags']]

In [29]:
predictions2['image_name'] = predictions2['image_name'].apply(lambda x: x.split('.')[0])
predictions2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,image_name,tags
0,test_16476,agriculture clear primary
1,test_15164,clear primary
2,test_16984,agriculture clear primary
3,test_4415,partly_cloudy primary
4,test_5227,agriculture clear habitation primary road
...,...,...
61186,file_12356,clear primary
61187,file_7191,agriculture clear habitation primary road
61188,file_17520,agriculture clear primary road
61189,file_4931,agriculture clear habitation primary road


In [30]:
submission = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
submission

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water
...,...,...
61186,file_9995,primary clear agriculture road water
61187,file_9996,primary clear agriculture road water
61188,file_9997,primary clear agriculture road water
61189,file_9998,primary clear agriculture road water


In [31]:
new_submissions = pd.merge(submission, predictions2, on='image_name', how='left')

In [32]:
new_submissions.columns = ['image_name', 'remove', 'tags']
new_submission = new_submissions.drop(columns= 'remove')

In [34]:
new_submission.to_csv('submission.csv', index=False)

![alt text](competitionscore.png 'Title')

![img](second.png)